In [1]:
import torch
import torch.nn as nn
from vit_pytorch import ViT
from graphs import Graph, prims
import os
import numpy as np
import torchvision.models as models
import torchvision.transforms as transforms
from torchvision.models import ResNet18_Weights
from tree_dataset import TreeDataset
import model as m
from torch.utils.data import DataLoader
from d2lvit import *
import copy
from collections import OrderedDict

In [2]:
model = torch.load(os.path.join('..', 'models', 'finetune_3', 'digit-model.pt'))

In [3]:
train_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_4k'), m.resnet_preprocess()) 
val_set = TreeDataset(os.path.join('..', 'data', 'extra_variety_2k'), m.resnet_preprocess()) 
print(f'Train size: {len(train_set)} Val size: {len(val_set)}')
train_loader = DataLoader(train_set, batch_size=32)
val_loader = DataLoader(val_set, batch_size=32)
device = m.get_device()
config = {'labels_key': 'digit_labels'}

Train size: 4000 Val size: 2000
Identified CUDA device: NVIDIA GeForce RTX 3060


In [4]:
train_acc = m.predict(model, train_loader, device, config, None)
print(train_acc)

1.0


In [5]:
val_acc = m.predict(model, val_loader, device, config, None)
print(val_acc)

1.0


In [6]:
digits_model = copy.deepcopy(model)

In [7]:
model.heads = nn.Sequential(OrderedDict([('head', nn.Linear(768, 45))]))
model = model.to(device)

In [8]:
model(torch.unsqueeze(train_set[0]['image'], 0).to(device))

tensor([[ 0.1761,  0.0646, -0.4344,  0.2091, -0.7125, -0.3259,  0.4421, -0.7209,
          0.6050,  0.1678, -0.2778,  0.2430, -0.1893, -0.0903, -1.0688,  0.4158,
         -1.0061, -0.0751, -0.8144,  0.0921, -0.4842, -0.5707, -0.0627,  0.6933,
         -1.3600, -0.0814,  0.3077,  0.1235,  0.0099, -0.0283,  1.0226, -0.4961,
         -0.2277, -0.0533, -0.1118, -0.5529, -0.7020,  0.3817,  0.4434,  0.1478,
          0.2748,  0.6021,  0.4181,  0.0417, -0.2478]], device='cuda:0',
       grad_fn=<AddmmBackward0>)

In [9]:
m.train(model, 0.0001, 0, 100, train_loader, val_loader, device, os.path.join('..', 'models', 'finetune_noprims'), digits_model)

Epoch 10 done, train loss: 0.0006 val acc: 1.0000
Epoch 20 done, train loss: 0.0001 val acc: 1.0000
Epoch 30 done, train loss: 0.0001 val acc: 1.0000
Epoch 40 done, train loss: 0.0000 val acc: 1.0000
Epoch 50 done, train loss: 0.0000 val acc: 1.0000
Epoch 60 done, train loss: 0.0000 val acc: 1.0000
Epoch 70 done, train loss: 0.0000 val acc: 1.0000
Epoch 80 done, train loss: 0.0000 val acc: 1.0000
Epoch 90 done, train loss: 0.0000 val acc: 1.0000
Epoch 100 done, train loss: 0.0000 val acc: 1.0000


In [10]:
model = torch.load(os.path.join('..', 'models', 'finetune_noprims', 'tree-model.pt'))

In [11]:
config = {'labels_key': 'tree_label'}
train_acc = m.predict(model, train_loader, device, config, digits_model)
print(train_acc)

1.0


In [12]:
val_acc = m.predict(model, val_loader, device, config, digits_model)
print(val_acc)

1.0
